#Leaderboard version - delete at the end; keep the bottom part only

Instructions:

the pickle file stores 3 separate betas for each subject

In [ ]:
import scipy.io, scipy.interpolate
import scipy.signal
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
from scipy.stats import pearsonr
import pickle

In [ ]:
# # Load the hidden test set
# truetest_data = scipy.io.loadmat("truetest_data.mat")
# truetest_data = scipy.io.loadmat("truetest_data.mat")['truetest_data']

In [ ]:
# Load the leaderboard data
truetest_data = scipy.io.loadmat("leaderboard_data.mat")
LBecog = truetest_data['leaderboard_ecog']

OSError: could not read bytes

In [ ]:
# Load the pre-trained linear filter
with open('allBetas.pkl', 'rb') as f:
    allBetas  = pickle.load(f)

In [ ]:
print(allBetas[0].shape)
print(allBetas[1].shape)
print(allBetas[2].shape)

In [ ]:
def gen_windows(data, window_length, window_overlap, fs=1000):
    window_length_samples = int(window_length * fs / 1000)
    window_overlap_samples = int(window_overlap * fs / 1000)
    num_windows = (data.shape[0] - window_length_samples) // (window_length_samples - window_overlap_samples) + 1
    feature_windows = np.zeros((num_windows, window_length_samples, data.shape[1]))
    for i in range(num_windows):
        start_idx = i * (window_length_samples - window_overlap_samples)
        end_idx = start_idx + window_length_samples
        feature_windows[i] = data[start_idx:end_idx]
    return feature_windows

def extract_features(window, fs=1000):
    features = []
    mean_voltage = np.mean(window, axis=0)
    features.append(mean_voltage)
    nperseg = window.shape[0]
    f, Pxx = scipy.signal.welch(np.transpose(window), fs=fs, nperseg=nperseg)
    bands = [(5, 15), (20, 25), (75, 115), (125, 160), (160, 175)]
    for low, high in bands:
        mask = (f >= low) & (f <= high)
        band_power = np.mean(Pxx[:, mask], axis=1) if np.any(mask) else np.zeros(Pxx.shape[0])
        features.append(band_power)
    return np.concatenate(features)

def cubic_spline_interpolation(y, old_freq, new_freq, total_duration):
    old_times = np.linspace(0, total_duration, int(total_duration*old_freq))
    new_times = np.linspace(0, total_duration, int(total_duration*new_freq))
    new_y = np.zeros((int(total_duration*new_freq), 5))
    for columnIndex in range(y.shape[1]):
      interpolator = scipy.interpolate.CubicSpline(old_times, y[:, columnIndex], bc_type='clamped')
      new_y[:, columnIndex] = interpolator(new_times)
    print(new_y.shape)
    return new_times, new_y

def pad_data(interpolated_data, original_length):
    pad_before = (original_length - len(interpolated_data)) // 2
    pad_after = original_length - len(interpolated_data) - pad_before
    padded_data = np.pad(interpolated_data, ((pad_before, pad_after), (0,0)), mode='constant')
    return padded_data

def moving_average_interpolated(data, window_size):
    num_rows, num_cols = data.shape
    smoothed_data = np.zeros_like(data)
    for col in range(num_cols):
        smoothed_data[:, col] = np.convolve(data[:, col], np.ones(window_size) / window_size, mode='same')
    return smoothed_data

In [ ]:
# replace with hidden test set for final submission
predicted_dg_python = {}

for i in range(len(truetest_data['leaderboard_ecog'])):  # Replace with actual leaderboard data or hidden test set
    window_length = 100
    window_overlap = 50
    pred_windows = gen_windows(truetest_data['leaderboard_ecog'][i][0], window_length, window_overlap)  # Replace with actual leaderboard data or hidden test set

    # Generate feature windows and extract features for prediction
    pred_features = np.array([extract_features(window) for window in pred_windows])

    lagVal = 3  # Make sure this matches the lag value used during training
    X_pred = np.array([pred_features[i:i+lagVal].flatten() for i in range(len(pred_features) - lagVal + 1)])

    print("X_pred shape:", X_pred.shape)
    print("beta shape:", allBetas[i])

    # Generate predictions using the pre-trained linear filter
    roughPreds = np.matmul(X_pred, allBetas[i])

    allPreds = moving_average_interpolated(roughPreds, 8)

    # Interpolate and structure for submission
    original_length = len(truetest_data['leaderboard_ecog'][i][0])  # Replace with actual leaderboard data or hidden test set
    total_duration = len(allPreds) * 0.05

    _, interpolated_predictions = cubic_spline_interpolation(allPreds, old_freq=20, new_freq=1000, total_duration=total_duration)
    padded_predictions = pad_data(interpolated_predictions, original_length)

    predicted_dg_python[f"Subject {i+1}"] = padded_predictions

# Convert the dictionary to a MATLAB cell array structure
# predicted_dg = {'cell_array': scipy.io.matlab.mio5_params.mat_struct()}
# for i in range(len(predicted_dg_python)):
#     predicted_dg['cell_array'].__dict__[f"Subject {i+1}"] = predicted_dg_python[f"Subject {i+1}"]

# # Save the predictions to 'predicted_dg.mat'
# scipy.io.savemat('predicted_dg.mat', predicted_dg)

# Convert the dictionary to a MATLAB cell array structure
predicted_dg = np.zeros((1, len(predicted_dg_python)), dtype=object)
for i in range(len(predicted_dg_python)):
    predicted_dg[0, i] = predicted_dg_python[f"Subject {i+1}"]

predicted_dg_dict = {'predicted_dg': predicted_dg}

# Save the predictions to 'predicted_dg.mat'
scipy.io.savemat('predicted_dg.mat', predicted_dg_dict)

In [ ]:
# Load the saved predicted_dg.mat file
loaded_data = scipy.io.loadmat('predicted_dg.mat')

# Access the predicted_dg variable from the loaded data
predicted_dg = loaded_data['predicted_dg']

# Print the shape and data type of predicted_dg
print("Shape of predicted_dg:", predicted_dg.shape)
print("Data type of predicted_dg:", predicted_dg.dtype)

# Iterate over each subject in predicted_dg
for i in range(predicted_dg.shape[1]):
    subject_data = predicted_dg[0, i]
    print(f"\nSubject {i+1}:")
    print("Shape of subject data:", subject_data.shape)
    print("Data type of subject data:", subject_data.dtype)

    # Print a sample of the subject data
    print("Sample of subject data:")
    print(subject_data[:5, :])

#Final Algorithm Submission

##Instructions

Upload the following: truetest_data.mat (the hidden test set), allBetas.pkl (the pre-trained linear filter), and run the Final_Format.ipynb script.

The script will load the hidden test set, apply the pre-trained linear filter, generate predictions, and save the predictions to a file named predicted_dg.mat.

Download the predicted_dg.mat file from Google Colab, which contains the predicted finger flexion values for the hidden test set.



In [ ]:
import scipy.io, scipy.interpolate
import scipy.signal
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
from scipy.stats import pearsonr
import pickle

# Load the hidden test set
truetest_data = scipy.io.loadmat("truetest_data.mat")

# Load the pre-trained linear filter
with open('allBetas.pkl', 'rb') as f:
    allBetas  = pickle.load(f)

print(allBetas[0].shape)
print(allBetas[1].shape)
print(allBetas[2].shape)

def gen_windows(data, window_length, window_overlap, fs=1000):
    window_length_samples = int(window_length * fs / 1000)
    window_overlap_samples = int(window_overlap * fs / 1000)
    num_windows = (data.shape[0] - window_length_samples) // (window_length_samples - window_overlap_samples) + 1
    feature_windows = np.zeros((num_windows, window_length_samples, data.shape[1]))
    for i in range(num_windows):
        start_idx = i * (window_length_samples - window_overlap_samples)
        end_idx = start_idx + window_length_samples
        feature_windows[i] = data[start_idx:end_idx]
    return feature_windows

def extract_features(window, fs=1000):
    features = []
    mean_voltage = np.mean(window, axis=0)
    features.append(mean_voltage)
    nperseg = window.shape[0]
    f, Pxx = scipy.signal.welch(np.transpose(window), fs=fs, nperseg=nperseg)
    bands = [(5, 15), (20, 25), (75, 115), (125, 160), (160, 175)]
    for low, high in bands:
        mask = (f >= low) & (f <= high)
        band_power = np.mean(Pxx[:, mask], axis=1) if np.any(mask) else np.zeros(Pxx.shape[0])
        features.append(band_power)
    return np.concatenate(features)

def cubic_spline_interpolation(y, old_freq, new_freq, total_duration):
    old_times = np.linspace(0, total_duration, int(total_duration*old_freq))
    new_times = np.linspace(0, total_duration, int(total_duration*new_freq))
    new_y = np.zeros((int(total_duration*new_freq), 5))
    for columnIndex in range(y.shape[1]):
      interpolator = scipy.interpolate.CubicSpline(old_times, y[:, columnIndex], bc_type='clamped')
      new_y[:, columnIndex] = interpolator(new_times)
    print(new_y.shape)
    return new_times, new_y

def pad_data(interpolated_data, original_length):
    pad_before = (original_length - len(interpolated_data)) // 2
    pad_after = original_length - len(interpolated_data) - pad_before
    padded_data = np.pad(interpolated_data, ((pad_before, pad_after), (0,0)), mode='constant')
    return padded_data

def moving_average_interpolated(data, window_size):
    num_rows, num_cols = data.shape
    smoothed_data = np.zeros_like(data)
    for col in range(num_cols):
        smoothed_data[:, col] = np.convolve(data[:, col], np.ones(window_size) / window_size, mode='same')
    return smoothed_data

# replace with hidden test set for final submission
predicted_dg_python = {}

for i in range(len(truetest_data)):
    window_length = 100
    window_overlap = 50
    pred_windows = gen_windows(truetest_data[i][0], window_length, window_overlap)

    # Generate feature windows and extract features for prediction
    pred_features = np.array([extract_features(window) for window in pred_windows])

    lagVal = 3  # Make sure this matches the lag value used during training
    X_pred = np.array([pred_features[i:i+lagVal].flatten() for i in range(len(pred_features) - lagVal + 1)])

    print("X_pred shape:", X_pred.shape)
    print("beta shape:", allBetas[i].shape)

    # Generate predictions using the pre-trained linear filter
    roughPreds = np.matmul(X_pred, allBetas[i])

    allPreds = moving_average_interpolated(roughPreds, 8)

    # Interpolate and structure for submission
    original_length = len(truetest_data[i][0])
    total_duration = len(allPreds) * 0.05

    _, interpolated_predictions = cubic_spline_interpolation(allPreds, old_freq=20, new_freq=1000, total_duration=total_duration)
    padded_predictions = pad_data(interpolated_predictions, original_length)

    predicted_dg_python[f"Subject {i+1}"] = padded_predictions

# Convert the dictionary to a MATLAB cell array structure
predicted_dg = {'cell_array': scipy.io.matlab.mio5_params.mat_struct()}
for i in range(len(predicted_dg_python)):
    predicted_dg['cell_array'].__dict__[f"Subject {i+1}"] = predicted_dg_python[f"Subject {i+1}"]

# Save the predictions to 'predicted_dg.mat'
scipy.io.savemat('predicted_dg.mat', predicted_dg)

In [ ]:
# suggested modif
# Convert the dictionary to a MATLAB cell array structure
predicted_dg = np.zeros((1, len(predicted_dg_python)), dtype=object)
for i in range(len(predicted_dg_python)):
    predicted_dg[0, i] = predicted_dg_python[f"Subject {i+1}"]

predicted_dg_dict = {'predicted_dg': predicted_dg}

# Save the predictions to 'predicted_dg.mat'
scipy.io.savemat('predicted_dg.mat', predicted_dg_dict)